In [3]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 42.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 37.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 28.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sodapy import Socrata
from requests.exceptions import ReadTimeout, ConnectTimeout
import time

# Define your Socrata credentials (app token is optional but recommended for higher rate limits)
app_token = None # Replace with your app token or leave as an empty string
client = Socrata("data.cityofnewyork.us", app_token, timeout=120)  # Set a 120-second timeout

# Define the dataset identifier and query parameters
dataset_identifier = "wvxf-dwi5"
where_clause = "ViolationStatus='Close' AND (class='B' OR class='c') AND novissueddate > '2023-07-19T00:00:00'"
batch_size = 10000  # Fetch data in smaller batches
offset = 0  # Start with offset 0
all_results = []  # Initialize an empty list to store all results

# Function to fetch data with retry mechanism
def fetch_data_with_retry(client, dataset_identifier, where_clause, limit, offset, retries=3):
    print(offset)
    for attempt in range(retries):
        try:
            # Fetch the data using the Socrata client
            results = client.get(dataset_identifier, where=where_clause, limit=limit, offset=offset)
            return results
        except (ReadTimeout, ConnectTimeout) as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                print("Retrying...")
                time.sleep(5)  # Wait for 5 seconds before retrying
            else:
                print("Max retries reached. Exiting.")
                raise

# Fetch data in batches until all records are retrieved
while True:
    results = fetch_data_with_retry(client, dataset_identifier, where_clause, batch_size, offset)
    if not results:
        break  # No more results to fetch
    all_results.extend(results)
    offset += batch_size

# Convert the results to a pandas DataFrame
df = pd.DataFrame.from_records(all_results)

# Display the DataFrame (optional)
print(df)

# Optionally save DataFrame to a CSV file
df.to_csv("results.csv", index=False)


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
